# Weather Patterns Around the Equator 
---
## Observations

---
### Set Up

In [1]:
# Dependencies
import pandas as pd
import requests
from datetime import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import api_keys
import json
from pprint import pprint
from citipy import citipy
from pandas.io.json import json_normalize

In [2]:
# location for save file
now = datetime.now()
savefile = "Output/" + now.strftime("%Y-%m-%d") + "_WeatherSample.csv"
savefile

'Output/2018-09-28_WeatherSample.csv'

In [3]:
# Coordinate Limits
lat_range = (-90.0, 90.0)
lon_range = (-180.0, 180.0)

# Sample size variables
target_size = 500
land_pct = .29
pool = round(target_size / land_pct)

# Container for sample cities
city_list = []

# Url for API calls
f = "imperial"   
c = "metric"
k = "standard"
owm_url = (f"http://api.openweathermap.org/data/2.5/weather?appid={api_keys.owm}&units={f}&q=")

---
### ID Locations for Testing

In [4]:
def rand_cities(size):
    # Generate random numbers for lat/lon coords
    rand_lat = np.random.uniform(lat_range[0], lat_range[1], size=size)
    rand_lon = np.random.uniform(lon_range[0], lon_range[1], size=size)
    
    # Container for return values
    results = []
    
    # Get our sample of cities
    for i in np.arange(size):
    
        # Find the closest city to our coord pairs
        city = citipy.nearest_city(rand_lat[i], rand_lon[i]).city_name
    
        # Add to list if it's unique
        if city not in results:
            results.append(city)
            
    # Return the resulting find        
    return results

In [5]:
# Find at least 500 random unique cities to test on
while(len(city_list) < 500):
    city_list = rand_cities(pool)
    
# Check the size of our sample
city_count = len(city_list)
print(f"{city_count} unique cities out of {pool} random queries found!")

704 unique cities out of 1724 random queries found!


In [6]:
for i in city_list:
    print(i)

bauchi
saint george
pevek
bethel
baruun-urt
vizinga
vila franca do campo
khatanga
sao domingos do maranhao
cabra
ivanivka
andros town
thunder bay
cidreira
san patricio
bandarbeyla
leningradskiy
hamilton
zhigansk
tevaitoa
severo-kurilsk
taolanaro
sentyabrskiy
cagayan de tawi-tawi
lavrentiya
rikitea
saskylakh
nikolskoye
albany
jamestown
berlevag
castro
vicksburg
chihuahua
zhemchuzhnyy
bozoum
saint-leu
kapaa
mar del plata
eyrarbakki
faanui
montevideo
barrow
ushuaia
puerto del rosario
cape town
qaanaaq
mahebourg
jalu
nizhneyansk
marsaxlokk
amderma
tongren
shimanovsk
tasiilaq
tiksi
tadine
cordoba
east london
port alfred
jizan
samusu
nanortalik
hermanus
sorong
owando
gazimagusa
georgetown
thompson
carnarvon
talnakh
nizhniy odes
san carlos de bariloche
mayor pablo lagerenza
vaini
olinda
koumac
vestmannaeyjar
monrovia
tuktoyaktuk
gillette
qasigiannguit
butaritari
tumannyy
taoudenni
saldanha
punta arenas
comodoro rivadavia
westport
mataura
anshun
barbar
joshimath
faya
saint-philippe
solnechnyy


---
### Check Weather for Locations

In [7]:
response = requests.get(owm_url+city_list[0])

In [8]:
response

<Response [200]>

In [9]:
weather = response.json()
pprint(weather)

{'base': 'stations',
 'clouds': {'all': 80},
 'cod': 200,
 'coord': {'lat': 10.31, 'lon': 9.85},
 'dt': 1538167975,
 'id': 2347470,
 'main': {'grnd_level': 951.06,
          'humidity': 83,
          'pressure': 951.06,
          'sea_level': 1024.64,
          'temp': 72.82,
          'temp_max': 72.82,
          'temp_min': 72.82},
 'name': 'Bauchi',
 'sys': {'country': 'NG',
         'message': 0.0051,
         'sunrise': 1538111363,
         'sunset': 1538154747},
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 152.502, 'speed': 3.6}}


In [10]:
df_test = pd.DataFrame(json_normalize(weather))
df_test

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,80,200,10.31,9.85,1538167975,2347470,951.06,83,951.06,...,72.82,72.82,Bauchi,NG,0.0051,1538111363,1538154747,"[{'id': 803, 'main': 'Clouds', 'description': ...",152.502,3.6


In [11]:
def status_frmct(position, size):
    diff = len(str(size)) - len(str(position))
    result = ""
    
    for i in np.arange(diff):
        result += " "
    
    return result + (f"{position}/{size}")

In [12]:
def print_last(queue, number):
    last = len(queue)
    for i in np.arange(last-number, last):
        print(queue[i])
    return

In [13]:
# Log the status of the API calls
separator = "--------------------------------------------------"
api_call_log = ["Attempting to Retrieve data...", separator]
print_last(api_call_log, 2)

# Variables for helping with tracking/containing call results
attempt = 1
found = 0
weather_list = []

# Run an API call for each city in our sample
for city in city_list:
    status = (f"Attempt on call {status_frmct(attempt,city_count)}: {city.title()}   >>>   ")
    #response = requests.get(owm_url + city)
    response = [200]
    
    if (response[0] == 200):
        found += 1
        status += "Success."
        #weather_list.append(response.json())
    else:
        status += (f"Error {response[0]}!")
    
    # Print a status update for each city
    api_call_log.append(status)
    print(status)
    
    # Create a delay so that the max number of openweathermap api calls is not exceeded (60/min)
    if((attempt%60) == 0):
        api_call_log.append(separator)
        api_call_log.append("Maximum number of calls per minute in OpenWeatherMap.org made.  Waiting for cooldown.")
        api_call_log.append(separator)
        print_last(api_call_log, 3)
        time.sleep(5)
        
    attempt += 1

api_call_log.append(separator)
api_call_log.append(f"Calls completed! Found {found} out of {city_count} records.")
print_last(api_call_log, 2)

Attempting to Retrieve data...
--------------------------------------------------
Attempt on call   1/704: Bauchi   >>>   Success.
Attempt on call   2/704: Saint George   >>>   Success.
Attempt on call   3/704: Pevek   >>>   Success.
Attempt on call   4/704: Bethel   >>>   Success.
Attempt on call   5/704: Baruun-Urt   >>>   Success.
Attempt on call   6/704: Vizinga   >>>   Success.
Attempt on call   7/704: Vila Franca Do Campo   >>>   Success.
Attempt on call   8/704: Khatanga   >>>   Success.
Attempt on call   9/704: Sao Domingos Do Maranhao   >>>   Success.
Attempt on call  10/704: Cabra   >>>   Success.
Attempt on call  11/704: Ivanivka   >>>   Success.
Attempt on call  12/704: Andros Town   >>>   Success.
Attempt on call  13/704: Thunder Bay   >>>   Success.
Attempt on call  14/704: Cidreira   >>>   Success.
Attempt on call  15/704: San Patricio   >>>   Success.
Attempt on call  16/704: Bandarbeyla   >>>   Success.
Attempt on call  17/704: Leningradskiy   >>>   Success.
Attempt on

Attempt on call 181/704: Dunedin   >>>   Success.
Attempt on call 182/704: Plastun   >>>   Success.
Attempt on call 183/704: Boa Vista   >>>   Success.
Attempt on call 184/704: Iqaluit   >>>   Success.
Attempt on call 185/704: Sibolga   >>>   Success.
Attempt on call 186/704: Puerto Escondido   >>>   Success.
Attempt on call 187/704: Korla   >>>   Success.
Attempt on call 188/704: Mutsamudu   >>>   Success.
Attempt on call 189/704: Flinders   >>>   Success.
Attempt on call 190/704: Medeiros Neto   >>>   Success.
Attempt on call 191/704: Jahrom   >>>   Success.
Attempt on call 192/704: Fort Nelson   >>>   Success.
Attempt on call 193/704: Mogotes   >>>   Success.
Attempt on call 194/704: Sar-E Pul   >>>   Success.
Attempt on call 195/704: Jackson   >>>   Success.
Attempt on call 196/704: Qusar   >>>   Success.
Attempt on call 197/704: Zumarraga   >>>   Success.
Attempt on call 198/704: Ishigaki   >>>   Success.
Attempt on call 199/704: Izumo   >>>   Success.
Attempt on call 200/704: Ola

Attempt on call 361/704: Coquimbo   >>>   Success.
Attempt on call 362/704: Griffith   >>>   Success.
Attempt on call 363/704: York   >>>   Success.
Attempt on call 364/704: Javanrud   >>>   Success.
Attempt on call 365/704: Jishou   >>>   Success.
Attempt on call 366/704: Cayenne   >>>   Success.
Attempt on call 367/704: Padang   >>>   Success.
Attempt on call 368/704: Young   >>>   Success.
Attempt on call 369/704: Prieska   >>>   Success.
Attempt on call 370/704: Miles City   >>>   Success.
Attempt on call 371/704: Loubomo   >>>   Success.
Attempt on call 372/704: Pucallpa   >>>   Success.
Attempt on call 373/704: Clyde River   >>>   Success.
Attempt on call 374/704: Kununurra   >>>   Success.
Attempt on call 375/704: Lesnoye   >>>   Success.
Attempt on call 376/704: Homer   >>>   Success.
Attempt on call 377/704: Praia Da Vitoria   >>>   Success.
Attempt on call 378/704: Ionia   >>>   Success.
Attempt on call 379/704: Tirumullaivasal   >>>   Success.
Attempt on call 380/704: Awbari

Attempt on call 541/704: Dingle   >>>   Success.
Attempt on call 542/704: Chyhyryn   >>>   Success.
Attempt on call 543/704: Tunduru   >>>   Success.
Attempt on call 544/704: Sri Aman   >>>   Success.
Attempt on call 545/704: Cuxhaven   >>>   Success.
Attempt on call 546/704: Challakere   >>>   Success.
Attempt on call 547/704: Rungata   >>>   Success.
Attempt on call 548/704: Kontagora   >>>   Success.
Attempt on call 549/704: Toliary   >>>   Success.
Attempt on call 550/704: Coxim   >>>   Success.
Attempt on call 551/704: Poronaysk   >>>   Success.
Attempt on call 552/704: Ilam   >>>   Success.
Attempt on call 553/704: Beringovskiy   >>>   Success.
Attempt on call 554/704: Kyshtovka   >>>   Success.
Attempt on call 555/704: Katsuura   >>>   Success.
Attempt on call 556/704: Lohr   >>>   Success.
Attempt on call 557/704: Hailar   >>>   Success.
Attempt on call 558/704: Keffi   >>>   Success.
Attempt on call 559/704: Peniche   >>>   Success.
Attempt on call 560/704: Zlatoustovsk   >>> 

In [14]:
# Test
nums = [11122333,66444555, 1232233, 23423523, 3213223, 23324325]
yticks = ["$"+str(round((i/1000000), 2)) for i in nums]
yticks

['$11.12', '$66.44', '$1.23', '$23.42', '$3.21', '$23.32']

---
### Clean Up Data and Save to File

In [15]:
# Save findings to csv

---
### Data Visualization